In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
from evaluator.CodeBLEU import calc_code_bleu
import numpy as np
import os
from evaluator.bleu import _bleu
from utils import read_examples, check
from fuzzywuzzy import fuzz
%matplotlib inline
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
import ast
import astor
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")
loc = "t5_full4_dist_imp"
eval_examples = read_examples("/home/ubuntu/callargs/pkl_data/test.pkl", -1)

wh = "best-ppl"
a = f"/home/ubuntu/results/{loc}/test_{wh}.output"
b = f"/home/ubuntu/results/{loc}/test_{wh}.gold"
pred_nls = [x.strip() for x in open(a).readlines()]
gold = [x.strip() for x in open(b).readlines()]  

/home/ubuntu/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def sim_acc(pred_nls, gold, eval_examples):
    accs = []
    sims = []
    for x, y, ex in zip(pred_nls, gold, eval_examples):
        res = check(x.strip()) == check(y.strip())
        sim = fuzz.ratio(check(x.strip()), check(y.strip()))
        accs.append(res)
        sims.append(sim)
    return np.array(accs), np.array(sims)
accs, sims = sim_acc(pred_nls, gold, eval_examples)


In [3]:
import pickle
path = "/home/ubuntu/callargs/pkl_data/"
dic = {}
with open(f"{path}/train.pkl", "rb") as f:
    dic["train"] = pickle.load(f)
with open(f"{path}/dev.pkl", "rb") as f:
    dic["dev"] = pickle.load(f)
with open(f"{path}/test.pkl", "rb") as f:
    dic["test"] = pickle.load(f)

In [4]:
def lastn(text, num=-10):
    lis = text.split("\n")[num:]
    return '\n'.join(lis)

def sim(a, b):
    x = set(a.split())
    y = set(b.split())
    return len(x&y)/len(x)

threshold = 0.5
nwa = np.zeros(len(accs))
nws = np.zeros(len(sims))
mask = np.zeros(len(accs))
for i, x in enumerate(dic["test"]):
    if len(x["usages"])==0: continue
    if sim(lastn(x["context"][0]), lastn(x["usages"][0][0]))>threshold:
        mask[i]=1
        a = check(x["target"].strip())
        b = check(x["usages"][0][-1].strip())
        nwa[i] = a==b
        nws[i] =fuzz.ratio(a,b)
print(np.mean(accs), np.mean(sims))
print(np.mean(accs*(1-mask)+mask*nwa), np.mean(sims*(1-mask)+mask*nws))

0.5531017622491596 80.08733829072018
0.5777936233065091 81.71184679637364


In [5]:
parta = []
parts = []
for i, x in enumerate(dic["test"]):
    if len(x["usages"])==0: 
        parta.append(accs[i])
        parts.append(sims[i])
print(np.mean(parta), np.mean(parts))

0.6226367532140157 84.19527770775565
